In [24]:
def KDJ(data,N):
    import numpy as np 
    Lmax=data['Loprc'].rolling(N).min()
    Lmin=data['Hiprc'].rolling(N).max()
    RSV=(data['Clsprc'].values-Lmin)/(Lmax-Lmin)
    K=np.zeros((len(RSV)))
    D=np.zeros((len(RSV)))
    J=np.zeros((len(RSV)))
    for t in range(N,len(data)):
        if t==0:
            K[t]=RSV[t]
            D[t]=RSV[t]
        if t>0:
            K[t]=2/3*K[t-1]+1/3*RSV[t]
            D[t]=2/3*D[t-1]+1/3*K[t]
        J[t]=3*D[t]-2*K[t]
    return (K,D,J)

import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Bar,Line,Kline
data = pd.read_excel('data.xlsx')
kdj = KDJ(data,9)
k = kdj[0]
d = kdj[1]
j = kdj[2]
x_data = data['Trddt'].tolist()
y_data = data[['Opnprc', 'Clsprc', 'Loprc', 'Hiprc']].apply(lambda i: [i[0], i[1], i[2], i[3]], axis = 1).tolist()
c = (
    Kline()
    .add_xaxis(x_data[:50])
    .add_yaxis("kline", y_data[:50], yaxis_index = 1)
    .set_global_opts(
        xaxis_opts = opts.AxisOpts(is_scale = True),
        yaxis_opts = opts.AxisOpts(is_scale = True),
        title_opts = opts.TitleOpts(title="Kline-基本事例")
    )
    .extend_axis(yaxis = opts.AxisOpts(name = "k线", min_ = min(data.loc[:50, 'Loprc']), max_ = max(data.loc[:50, 'Hiprc'])))
)
line = Line()
line.add_xaxis(x_data[:50])
line.add_yaxis('k', k)
line.add_yaxis('d', d)
line.add_yaxis('j', j)
line.set_series_opts(opts.LabelOpts(is_show = False), markline_opts = opts.MarkLineOpts(data = [
        opts.MarkLineItem(y = 0.2, name = "超卖"),
        opts.MarkLineItem(y = 0.5, name = "震荡"),
        opts.MarkLineItem(y = 0.8, name = "超买"),
]))
c.overlap(line)

c.render_notebook()
